Para realizar a análise, regressão e classificação, escolhemos o dataset Housing Prices, disponível em:

https://www.kaggle.com/datasets/yasserh/housing-prices-dataset

O dataset é composto por apenas 1 arquivo .csv separado por vírgulas (",")
O arquivo contém 13 colunas, sendo o objetivo realizar a regressão para inferir o preço das casas.

1) importar as dependências necessárias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, RobustScaler

2) leitura do arquivo

In [ ]:
df = pd.read_csv('Housing.csv', delimiter=',', header=0)
df.info()

3) limpeza de dados indesejados

In [ ]:
df_cp = df.copy()
df_cp = df_cp.drop_duplicates()
obj_cols = [col for col in df_cp.columns if df_cp[col].dtype == 'object']
# para colunas de valor booleano (sim ou não) devem existir apenas 2 tipos de resposta, furnishingstatus deve possuir 3 valores
print(f"Valores repetidos:\n{df_cp[obj_cols].nunique()}")

# caso exista valores fora do padrão, devem ser substituidos
valores_aceitos = ['yes', 'no']
valores_furnishingstatus = ['furnished', 'semi-furnished', 'unfurnished']

for col in df_cp.columns:
    if col in obj_cols:
        if col == 'furnishingstatus':
            df_cp[col] = df_cp[col].apply(lambda x: x if x in valores_furnishingstatus else 'unfurnished')
        else:
            df_cp[col] = df_cp[col].apply(lambda x: x if x in valores_aceitos else 'no')
    else:
        df_cp[col] = df_cp[col].fillna(df_cp[col].median())


print(f"\n\nValores faltantes:\n{df_cp.isnull().sum()}")

4) Transformação dos dados de atributos qualitativos para quantitativos

In [ ]:
# transformação de dados qualitativos em quantitativos
le = LabelEncoder()
for col in obj_cols:
  df_cp[col] = le.fit_transform(df_cp[col])


5) mostrar a correlação entre colunas

In [ ]:
# plot do mapa de calor de correlação
corr_person = df_cp.corr(method='pearson')
corr_sperman = df_cp.corr(method='spearman')
corr_kendall = df_cp.corr(method='kendall')
corr_list = [corr_person, corr_sperman, corr_kendall]
for corr in corr_list:
  plt.figure(figsize=(8, 6))
  sns.heatmap(corr, cmap="YlGnBu", annot=True, fmt=".1f")

É possível verificar que Price possui índices de correlação rasoáveis.

No geral, isso pode implicar em boa acurácia para regressão do campo Price.

Para a redução de dimensionalidade, é escolhido remover o campo com menor correlação para as colunas alvo (hotwaterheating), que possui correlação 0.1 com o campo price.


In [ ]:
del df_cp['hotwaterheating']

6) Análise descritiva

In [ ]:
df_cp[["price","area"]].describe()

In [ ]:
colunas_alvo = ["price","area","bedrooms","bathrooms","stories"]
fig, axs = plt.subplots(ncols=5, figsize=(25,3))
for i, col in enumerate(colunas_alvo):
  axs[i].set_title(f'Distribuição de {col}')
  sns.histplot(df_cp, x=col, ax=axs[i])


Outro ponto é que as variáveis quantitativas do dataset seguem a distribuição normal, o que é um bom sinal e é esperado. A padronização é um bom cadidato para este conjunto de dados, por conta do que foi pontuado aqui.

7) Descobrir se existem outliers nas colunas

In [ ]:
colunas_alvo = ["price","area"]
fig, axs = plt.subplots(ncols=2, figsize=(12,4))
for i, col in enumerate(colunas_alvo):
  axs[i].set_title(f'Outliers de {col}')
  sns.boxplot(df_cp, y=col, ax=axs[i])


É possível perceber que existem outliers em ambas as colunas. Mas por serem valores aceitáveis, mesmo que altos, serão mantidos para a análise final.

Para fins de demonstração, o código para remoção dos outliers é implementado.

In [ ]:
colunas_alvo = ["price","area"]
df_clean = df_cp.copy()
for col in colunas_alvo:
  Q1 = df_clean[col].quantile(0.25)
  Q3 = df_clean[col].quantile(0.75)
  IQR = Q3 - Q1
  limite_inferior = Q1 - 1.5 * IQR
  limite_superior = Q3 + 1.5 * IQR
  df_clean = df_clean[(df_clean[col] >= limite_inferior) & (df_clean[col] <= limite_superior)]

fig, axs = plt.subplots(ncols=2, figsize=(12,4))
for i, col in enumerate(colunas_alvo):
  axs[i].set_title(f'Outliers de {col}')
  sns.boxplot(df_clean, y=col, ax=axs[i])

8) Normalização dos dados

In [ ]:
rs = RobustScaler()
X_stand = rs.fit_transform(df_cp)

df_rs = pd.DataFrame(X_stand, columns=['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad',
       'guestroom', 'basement', 'airconditioning', 'parking', 'prefarea',
       'furnishingstatus'])

# código para desfazer a transformação
# dados = rs.inverse_transform(df_rs)
# df_back = pd.DataFrame(dados, columns=['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad',
#        'guestroom', 'basement', 'airconditioning', 'parking', 'prefarea',
#        'furnishingstatus'])

df_rs.head()

9) divisão do dataset em conjunto de treinamento e conjunto de teste

In [ ]:
training = df_rs.sample(frac=0.8)
test = df_rs.drop(training.index)

training_y = training['price']
test_y = test['price']

del training['price']
del test['price']

Treinamento do algoritmo para price